# pega-pronunciamentos-html

 Esse script usa os csvs gerados em `pega-links` e extrai os pronunciamentos para os quais existem links para conteúdo em html.

#### TO DO:
- Implementar função `download_pdf()`

#### Importa pacotes

In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import pprint as pp
import os
import re
import requests
import time
from tqdm._tqdm_notebook import tqdm_notebook as tqdm

In [12]:
# Chama tqdm para monitorar progresso no pandas
tqdm.pandas()

#### Lê dados externos

In [13]:
def read_data(name, database):
    
    '''
    Lê arquivos .csv usando o pandas. Os arquivos devem
    seguir o padrão de nome '{nome-parlamentar}-{database}-metadada.csv'
    
    Parâmetros:
    
    name -> Nome do parlamentar separado por hífen (como "jair-bolsonaro")
    database -> "plenario" ou "comissao"
    '''
    
    fpath = "../data/tables/{name}-{database}-metadata.csv".format(
        name = name, 
        database = database
    )
    
    df = pd.read_csv(fpath, dtype='str')
    
    return df

#### Funções para extrair dados de um link e salvá-lo

In [14]:
def make_request(url):
    
    '''
    Recebe uma url, faz requisição get
    e retorna uma string de texto.
    '''

    headers = {"User-Agent":"Rodrigo Menegat, jornalista (rodrigoschuinski@gmail.com)"}
    r = requests.get(url, headers = headers)
    r.encoding = 'utf-8'
    return r.text

In [15]:
def make_soup(data):
    
    '''
    Recebe uma string de texto e retorna um objeto do BeautifulSoup.
    '''
    
    # Preciso retirar as tags </b> do texto porque
    # o desenvolvedor da Câmara colocou elas fora
    # de ordem e isso faz o raspador enlouquecer.
    #                 ¯\_(ツ)_/¯
    data = data.replace("<b>","").replace("</b>","")
    soup = BeautifulSoup(data, 'html.parser')
    return soup

In [16]:
def scrape_content_plen(soup):
    
    '''
    Usa o BeautifulSoup para extrair o conteúdo da URL
    em que está o discurso. Retorna uma string de texto.
    '''
        
    paragraph = soup.find( "p", attrs = {"align":"justify"} )
    
    # Esse passo a mais é necessário porque, em alguns casos, há mais de uma tag 'font' dentro do 'p'.
    # Nessas ocasiões, não era retornado todo o texto, mas apenas o conteúdo da primeira tag
    fonts = paragraph.find_all("font")
    
    text = [ font.text.strip() for font in fonts ]
    text = ' '.join(text)

    
    return text

In [17]:
def scrape_content_com(soup):
    
    '''
    Usa o BeautifulSoup para extrair o conteúdo da URL
    em que está o discurso. Retorna uma string de texto.
    '''
    
    fonts = soup.find_all("font")
    fonts = [ font.text.strip() for font in fonts ]
    content = ' '.join(fonts)
    
    return content

In [36]:
def save_txt(content, path, *args):
    
    '''
    Salva um arquivo txt no path designado.
    O array args é usado para gerar o nome do arquivo.
    '''
   
    file_id = '-'.join(args)
    file_id = file_id.replace("/","").replace(".","-")
    fname = path + file_id + ".txt"
    
    with open(fname, 'w+') as file:
        file.write(content)
        
    return

In [37]:
def download_pdf(url, date, path):
    
    '''
    Função para baixar o pdf do documento
    onde está registrado um pronunciamento.
    '''
    
    r = requests.get(url)
    
    # TO DO - implementar captura de sessões conjuntas,
    # cujo link leva para um PDF viewer feioso
    
    # Pega redirecionamentos
    if r.history:    
        # Acessa a tag de redirecionamento e pega o contúdo
        r = requests.get(url)
        soup = make_soup(r.text)
        
       # Acha o link de redirecionamento
        meta = soup.find('meta',attrs={'http-equiv':'Refresh'})
        meta = str(meta)
        meta = re.search(".*URL=(.*)\" http.*", meta)
        
        # Se encontrado, refaz a solicitação
        if meta:
            url = meta.group(1)
            r =  requests.get(url)
            
            # Seleciona página da declaração para salvar no nome do arquivo
            page_no = re.search("page=(\d+)", url).group(1)
            fpath = "{path}{date}-pg-{page_no}.pdf"
            fpath = fpath.format(path = path, 
                                 date = date, 
                                 page_no = page_no)
    
            if not os.path.isfile(fpath):
                with open(fpath, 'wb') as f:
                    f.write(r.content)
    
    else:
        print("Couldn't download the following url:\n" + url)
    
    return

#### Função que, via df.apply, as funções acima em cada linha do dataframe

In [38]:
def scrape_row_plen(row):
    
    '''
    Função para executar via df.apply()
    que acessa os dados de cada linha do
    dataframe e executa as operações ne-
    cessárias para acessar os dados. Usar
    apenas com a tabela com discursos feitos
    no plenário.
    '''
    
    if row.Discurso != '-':
    
        url = row.Discurso
        data = make_request(url)
        soup = make_soup(data)
        txt = scrape_content_plen(soup)
        
        path = "../data/txts/plenario/"
        file_id = [ str(row.name), row.Data, row.Hora, row.Fase, row.Sessão ]
        save_txt(txt, path, *file_id)
    
        # time.sleep(.2)
    
    else:
        url = row.Publicação
        path = "../data/pdfs/plenario/"
        date = row.Data.replace('/','-')
        # download_pdf(url, date, path)

        return
    

In [39]:
def scrape_row_com(row):

    '''
    Função para executar via df.apply()
    que acessa os dados de cada linha do
    dataframe e executa as operações ne-
    cessárias para acessar os dados. Usar
    apenas com a tabela com discursos feitos
    no plenário.
    '''
    
    url = row.Texto
    data = make_request(url)
    soup = make_soup(data)
    txt = scrape_content_com(soup)

    path = "../data/txts/comissao/"
    file_id = [row.Data, row.Hora, row.Reunião]
    save_txt(txt, path, *file_id)

    # time.sleep(.2)
    
    return

#### Roda funções

In [40]:
def scrape_df(df):
    
    '''
    Função que encapsula os métodos de raspagem aplicados via df.apply().
    Ela é útil para detectar com qual tabela (plenário ou comissões)
    estamos lidando e fazer a operação correta, sem precisar se preocupar
    com selecionar a função específica para o dataframe.
    '''

    if 'Sessão' in df.columns:
        df.progress_apply(scrape_row_plen, axis = 1)
    elif 'Comissão' in df.columns:
        df.progress_apply(scrape_row_com, axis = 1)
    else:
        raise Exception("O dataframe não está em um formato conhecido.")
        
    return

In [41]:
df_plen = read_data('jair-bolsonaro', 'plenario')

In [43]:
scrape_df(df_plen)

A Jupyter Widget

In [ ]:
#df_com = read_data('jair-bolsonaro', 'comissao')

In [ ]:
#scrape_df(df_com)